In [1]:
import asdf
import os
import numpy as np

from dataclasses import dataclass

In [2]:
@dataclass
class Ellipse:
    """An ellipse defined by semi-major and semi-minor axes.

    Note: Using a dataclass to define the object so that we get `==` for free.
    """

    semi_major: float
    semi_minor: float


ellipse_uri = "asdf://example.com/example-project/schemas/ellipse-1.0.0"

ellipse_schema_content = f"""
%YAML 1.1
---
$schema: http://stsci.edu/schemas/yaml-schema/draft-01
id: {ellipse_uri}

type: object
properties:
  semi_major:
    type: number
  semi_minor:
    type: number
required: [semi_major, semi_minor]
...
"""

ellipse_manifest_uri = "asdf://example.com/example-project/manifests/shapes-1.0.0"
ellipse_extension_uri = "asdf://example.com/example-project/extensions/shapes-1.0.0"
ellipse_tag = "asdf://example.com/example-project/tags/ellipse-1.0.0"

ellipse_manifest_content = f"""
%YAML 1.1
---
id: {ellipse_manifest_uri}
extension_uri: {ellipse_extension_uri}

title: Example Shape extension 1.0.0
description: Tags for example shape objects.

tags:
  - tag_uri: {ellipse_tag}
    schema_uri: {ellipse_uri}
...
"""


class EllipseConverter(asdf.extension.Converter):
    tags = [ellipse_tag]
    types = [Ellipse]

    def to_yaml_tree(self, obj, tag, ctx):
        return {
            "semi_major": obj.semi_major,
            "semi_minor": obj.semi_minor,
        }

    def from_yaml_tree(self, node, tag, ctx):
        return Ellipse(semi_major=node["semi_major"], semi_minor=node["semi_minor"])

# Adding your extensions to ASDF via Entry-Points

Obviously, having to dynamically add all the resources and extensions to ASDF every
time you want to work with a custom object is tedious. Thus ASDF uses Python entry-points
(mechanism for one python package to communicate information to another Python package),
to enable automatic discovery and loading of resources and extensions for ASDF.

Since entry-points are a means for python packages to communicate with one-another,
their use requires you to package your Python code, which is can be a complex issue.
Thus we will assume that you have an existing Python package, that you wish to add
our example ASDF extension to.

To create our entry-points we will need to make three modifications to the packaging
components of the existing Python package.
1. Include the resource `yaml` files into the Python package.
2. Create an entry point to add the resources to ASDF.
3. Create an entry point to add the extension to ASDF.

Note that we will assume that you are using the `setup.cfg` file to configure
your python package.

## Including the Resources in the Package

When your Python package is distributed on PyPi (or other services) it typically
only includes the Python files needed to operate your package. However, since
ASDF typically stores these files as `yaml` files, these files will not get
added to that packaged product by default. Thus one needs to configure the Python
package to specifically include these files. This can be done in several ways
one way to do this is by adding the following entry to the `setup.cfg` file:
```
[options.package_data]
* = *.yaml
```
However, other options exist such as using the builtin `importlib.resources` package.

## Create an Entry-Point for the Resources

ASDF treats the information it receives from the entry-points it checks for resources
as a function that it can evaluate to get a list of resource mappings. Suppose that
there your package is called `asdf_shapes` and the function you need to call in order
to get this list of mappings is called `get_resource_mappings` and is located in the
`integration` module, that is you need to import `get_resource_mappings` from
`asdf_shapes.integration`.  Thus you will need to add the following to your `setup.cfg`:

```
[options.entry_points]
asdf.resource_mappings =
    asdf_shapes_schemas = asdf_shapes.integration:get_resource_mappings
```

The entry-point ASDF checks for resources is `asdf.resource_mappings`, and your
entry into that entry-point needs to be some identifier for your package, in this case
`asdf_shapes_schemas`. The remaining portion represents module:function.

Now lets talk about how to create the `get_resource_mappings`. First, lets go ahead a
create the `yaml` files for the resources we used in our example in order to illustrate
an example organization of these resource files:

In [3]:
schema_root = "resources/schemas"
manifest_root = "resources/manifests"

os.makedirs(schema_root, exist_ok=True)
os.makedirs(manifest_root, exist_ok=True)

with open(f"{schema_root}/ellipse-1.0.0.yaml", "w") as f:
    f.write(ellipse_schema_content)

with open(f"{manifest_root}/shapes-1.0.0.yaml", "w") as f:
    f.write(ellipse_manifest_content)

Now that we have a directory structure for our resources, ASDF provides the
`asdf.resource.DirectoryResourceMapping` object to crawl resource directories
and add files those files to a mapping which can be used by ASDF. These mapping objects can
then be provided to ASDF via an entry point.

These objects require two input parameters:
1. A path to the root directory which contains the resources to be added.
2. The prefix that will be used together with the file names to generate the URI
for the resource in question.

There are some optional inputs:
1. `recursive`: (default `False`) which determines if the object will search recursively through
subdirectories.
2. `filename_pattern`: (default: `*.yaml`) Glob pattern for the files that should be added.
3. `stem_filename`: (default: `True`) determine if the file extension should be removed when creating
the URI.

Now (assuming that we have `schema_root` and `manifest_root` already defined), we can define:

In [4]:
# In module asdf_shapes.integration
def get_resource_mappings():
    schema_prefix = "asdf://example.com/example-project/schemas/"
    manifest_prefix = "asdf://example.com/example-project/manifests/"
    return [
        asdf.resource.DirectoryResourceMapping(schema_root, schema_prefix),
        asdf.resource.DirectoryResourceMapping(manifest_root, manifest_prefix)
    ]

Which can then be referenced by the entry-point. Note that for performance reasons,
we suggest you limit the top-level imports of the file(s) you load your entry points
from to as few as possible, going as far as deferring imports to inside the entry-point
functions when possible. This is because asdf will import all of these models immediately
when `asdf.open` is called meaning large imports will cause noticeable delays especially
when using the command-line interface.

## Create an Entry-Point for the Extensions

In a similar fashion to resources, ASDF assumes the entry-points it checks for extensions
as functions which return lists of `asdf.extension.Extension` objects. Thus lets assume
your function is called `get_extensions` and is in the `asdf_shapes.integration` module
alongside `get_resource_mappings`. Adding the entry-point for this would look something
like:

```
[options.entry_points]
asdf.extensions =
    asdf_shapes_extensions = asdf_shapes.integration:get_extensions
```

The entry-point ASDF checks for `Extensions` is `asdf.extensions`, with the rest
of the structure of the entry-point matching that of the one for resource mappings.

The structure of `get_extensions` will be very similar to that for `get_resource_mappings`:

In [5]:
# In module asdf_shapes.integration
def get_extensions():
    # import EllipseConverter inside this function
    return [
        asdf.extension.ManifestExtension.from_uri(
            ellipse_manifest_uri,
            converters=[EllipseConverter()]
        )
    ]

Once your package is installed with these changes, ASDF will automatically detect and use
your ASDF extension as needed in a seamless fashion.